In [1]:
import requests
from bs4 import BeautifulSoup
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import json


In [2]:
def get_vacancies_from_headhunter(query, num_pages=1):
    """
    Функция для сбора вакансий с сайта HeadHunter по запросу.
    :param query: Строка запроса (например, 'Python Developer')
    :param num_pages: Количество страниц для сбора вакансий
    :return: Список вакансий в формате JSON
    """
    base_url = 'https://hh.ru/search/vacancy'
    vacancies = []
    
    for page in range(num_pages):
        params = {'text': query, 'page': page}
        response = requests.get(base_url, params=params)
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for vacancy in soup.find_all('div', {'class': 'vacancy-serp-item'}):
            title = vacancy.find('a', {'class': 'bloko-link'}).text.strip()
            company = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info'}).text.strip()
            link = vacancy.find('a', {'class': 'bloko-link'})['href']
            
            vacancies.append({
                'title': title,
                'company': company,
                'link': link
            })
    
    return vacancies


In [3]:
# Пример данных для обучения модели
resume_texts = [
    "Experienced in Python and Machine Learning", 
    "Expert in AWS and cloud technologies", 
    "Skilled in data analysis and SQL"
]
job_descriptions = [
    "Looking for a Python developer with ML expertise", 
    "Hiring for an AWS specialist", 
    "Seeking a SQL data analyst"
]
labels = [1, 1, 1]  # Метки, где 1 означает соответствие, а 0 - несоответствие

# Преобразование текста в числовые векторы
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(resume_texts + job_descriptions).toarray()

# Размерность входного слоя
input_dim = X.shape[1]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, labels * 2, test_size=0.2, random_state=42)

# Модель
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=2, validation_data=(np.array(X_test), np.array(y_test)))


Epoch 1/10
2/2 [==============================] - 1s 201ms/step - loss: 0.6646 - accuracy: 1.0000 - val_loss: 0.6037 - val_accuracy: 1.0000
Epoch 2/10
2/2 [==============================] - 0s 28ms/step - loss: 0.6281 - accuracy: 1.0000 - val_loss: 0.5748 - val_accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 0s 34ms/step - loss: 0.5921 - accuracy: 1.0000 - val_loss: 0.5469 - val_accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 29ms/step - loss: 0.5580 - accuracy: 1.0000 - val_loss: 0.5191 - val_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 29ms/step - loss: 0.5247 - accuracy: 1.0000 - val_loss: 0.4919 - val_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 26ms/step - loss: 0.4912 - accuracy: 1.0000 - val_loss: 0.4650 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 23ms/step - loss: 0.4572 - accuracy: 1.0000 - val_loss: 0.4382 - val_accuracy: 1.0000
Epoch 8/10
2/2 [==========

In [4]:
def predict_vacancy_match(resume, vacancies):
    """
    Прогнозирует соответствие резюме и вакансий.
    :param resume: Текст резюме
    :param vacancies: Список вакансий
    :return: Список вакансий с их оценками соответствия
    """
    resume_vector = vectorizer.transform([resume]).toarray()
    results = []
    
    for vacancy in vacancies:
        vacancy_vector = vectorizer.transform([vacancy['title'] + ' ' + vacancy['company']]).toarray()
        match_score = model.predict(np.array([np.concatenate((resume_vector, vacancy_vector))]))
        results.append({
            'title': vacancy['title'],
            'company': vacancy['company'],
            'link': vacancy['link'],
            'match_score': match_score[0][0]
        })
    
    return results
